In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime
import re
from pathlib import Path

In [2]:
vox_url = 'https://www.vox.com/'
npr_url = 'https://www.npr.org/'

In [3]:
vox_res = requests.get(vox_url)
npr_res = requests.get(npr_url)

print(vox_res.status_code)
print(npr_res.status_code)

200
200


In [4]:
npr_soup = BeautifulSoup(npr_res.content, 'lxml')
vox_soup = BeautifulSoup(vox_res.content, 'lxml')

In [5]:
#whether the article was featured or not
headline = []
date_scraped = []
article_date = []
topic_tag = []
article_text = []
website = []

# NPR Scrape

Scraping Featured Stories
- url
- topic tag
- title

In [6]:
#NPR 

#find the section with the headline news and featured articles
group_section = npr_soup.find('div', {'class':'stories-wrap stories-wrap-featured'})

#get only articles
featured_stories = group_section.find_all('div', {'class':'story-wrap'})

npr_urls = []
#append urls for article 
for featured_story in featured_stories: 
    npr_urls.append(featured_story.find('a')['href'])          #get urls and append to urls list
    title = featured_story.find('h3', {'class': 'title'})  # find all titles
#     print(title, '\n')
    headline.append(title.text)                     #append titles to headline list
    slug = featured_story.find('h2', {'class':'slug'}).text #find section containing topic
#     print(slug.strip(), '\n\n')
    topic_tag.append(slug.strip())                   #append slug/topic to topic list
    
print(len(npr_urls))

<h3 class="title">The Postal Service is slowing the mail to save money. Critics say it's a death spiral</h3> 

Politics 


<h3 class="title">Journalists from the Philippines and Russia win the Nobel Peace Prize</h3> 

News 


<h3 class="title">U.S. officials will negotiate a new security deal with Mexico during talks Friday</h3> 

World 


<h3 class="title">No, it hasn't been the job recovery many had expected. But there's room for hope</h3> 

The Coronavirus Crisis 


<h3 class="title">Exclusive: Governors have questions about Afghan refugees. Here's who they call</h3> 

Politics 


<h3 class="title">California is the first state to ban 'stealthing,' nonconsensual condom removal</h3> 

Law 


<h3 class="title">'Black Lady Therapists' are still a thing on TV. But now, they're more interesting</h3> 

Pop Culture Happy Hour 


7


In [7]:
lists = [headline, date_scraped, article_date, topic_tag, article_text, website]

for item in lists:
    print(len(item))

7
0
0
7
0
0


Scraping Articles with One Article inside the story

- date of article
- full story
- date of scrape
- website

In [8]:
# print(url)
for url in npr_urls:
    page = requests.get(url)
    print(page.status_code)

    article_soup = BeautifulSoup(page.content, 'lxml')

    #section, including pictures that holds the paragraph tags
    text_section = article_soup.select_one('#storytext') # could I try rewriting as 
    if text_section is None:                      #article_soup.select_one(#storytext) ?? article_soup.find(div...)
        text_section = article_soup.find('div', {'class':'ArticlePage-articleBody'})
    
    # Find the images sections of the articles
    images = text_section.find_all('div', {'id': re.compile(r'res\d+')})
    for image in images: 
        image.extract()   #take out images from the text section due to nested 'p' tags

    # Grab the article text and append to the article list 
    full_story = ''
    for p in text_section.select('p'):
        full_story += p.text

    article_text.append(full_story)
#     print(full_story)
    article_date.append(article_soup.find('time')['datetime']) #time of article
    now = datetime.datetime.now() #time of scrape
    
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    #code from https://www.programiz.com/python-programming/datetime/current-datetime

    date_scraped.append(dt_string) #append to list 
    
    website.append('NPR')


200
200
200
200
200
200
200


In [9]:
lists = [headline, date_scraped, article_date, topic_tag, article_text, website]

for item in lists:
    print(len(item))

7
7
7
7
7
7


# Vox 

Scraping:
- Title

In [10]:
#find the section with the headline news and featured articles
group_section = vox_soup.find('div', {'class':'c-newspaper__main'})

#find the tag containing the titles for each featured article
titles = group_section.find_all('a', {'data-chorus-optimize-field':'hed'}) 

vox_urls = []
for title in titles:
    print(title.text)
    headline.append(title.text)  #append title to the headline list
    vox_urls.append(title['href']) #append url to a list 

The debt ceiling fight is far from over
The US is closer than ever before to making major progress on the climate crisis
How the risk of Covid-19 for kids compares to other dangers
The Supreme Court takes a lie detector test
The trillion-dollar coin scheme, explained by the guy who invented it 
Banks don’t want your money right now
Supply chain havoc is getting worse — just in time for holiday shopping
The WHO approval of the first malaria vaccine is a big deal


In [11]:
lists = [headline, date_scraped, article_date, topic_tag, article_text, website]

for item in lists:
    print(len(item))

15
7
7
7
7
7


- article_text
- date_scraped
- website
- topic_tag
- article_date
- website

In [12]:
for url in vox_urls:
    page = requests.get(url)
    print(page.status_code)
    article_soup = BeautifulSoup(page.content, 'lxml')

    
    text_section = article_soup.find('div', {'class':'c-entry-content'}) #isolate the text only from the article
    story_text = text_section.find_all('p') #grab all paragraph tags
    
    full_story = ''   #make an empty string to append each p to
    
    #get text under p tag and add to the full article story
    for p in story_text:
        full_story+= p.text
        
    article_text.append(full_story) #add to list of articles
    
    article_date.append(article_soup.find('time')['datetime']) #time of article
    now = datetime.datetime.now() #get current time of scrape
    
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    date_scraped.append(dt_string)
    #code from https://www.programiz.com/python-programming/datetime/current-datetime    
    website.append('Vox')  #record website of article origin
    
    topic_tag.append(None)


200
200
200
200
200
200
200
200


In [13]:
lists = [headline, date_scraped, article_date, topic_tag, article_text, website]

for item in lists:
    print(len(item))

15
15
15
15
15
15


# Create DataFrame 

In [14]:
vox_npr_df = pd.DataFrame({'headline':headline,
                              'date_scraped':date_scraped,
                              'article_date':article_date,
                              'topic_tag':topic_tag,
                              'article_text':article_text,
                              'website':website})

# Saving/Appending to CSV

In [15]:
file_name = Path("scraped_lib_articles.csv") #may need to change to os if we use s3 buckets

if file_name.exists():  #if file already exists in directory
    print('file exists, now appending')
    
    #append new scrapes to the file without header row
    vox_npr_df.to_csv('scraped_lib_articles.csv', mode='a', index=False, header=False) 

else: #if file does not exist yet
    print('file does not exist, now creating')
    vox_npr_df.to_csv('scraped_lib_articles.csv', index=False) #create new file 

file exists, now appending
